# Análisis y visualización de datos con python
# 4. Limpieza de datos

    - a) Tipos de datos
    - b) Eliminación de observaciones y variables
    - c) Valores faltantes
    - d) Limpieza de texto
    - e) Limpieza de categóricos
    - f) Limpieza de números
    - g) Limpieza de fechas
    - h) Ordenar y guardar datos

---

La limpieza de datos es el proceso de identificar, corregir o eliminar datos incorrectos, incompletos, irrelevantes o duplicados en un conjunto de datos. La limpieza de datos es importante para garantizar la precisión y la calidad de los datos y evitar errores en el análisis posterior. Algunas técnicas comunes de limpieza de datos incluyen la eliminación de valores atípicos, la eliminación de valores faltantes, la corrección de errores tipográficos y la eliminación de duplicados.

Empezaremos cargando los datos.

In [1]:
from numpy import nan
import pandas as pd

file_mfc = 'data_raw/Modulo-de-Fosas-Comunes_actualizacion24nov2022_VD.xlsx'
df = pd.read_excel(file_mfc, sheet_name="MFC", nrows=1000, index_col='ID' )
df.head()

,Estado_origen,Municipio_origen,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,,,,
XX-P001,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-30,2018-03-04 00:00:00,NaT,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23,NaN
XX-P002,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-12 00:00:00,NaT,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN
XX-P003,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-13 00:00:00,NaT,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN
XX-P004,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-14 00:00:00,NaT,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN
XX-P005,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-18 00:00:00,NaT,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN


Una consideración importante es si se modificara la columna original o se generará una columna nueva con los datos limpios. Qué opción se escoge depende de varios factores, cómo el tamaño de la tabla o si nos interesa conservar la información original. En este tutorial crearemos columnas nuevas para poder contrastar los cambios y al final quitaremos las que no nos interesen.


En este caso haremos una serie de correciones:
* Quitar columnas
* Cambiar tipos de datos
* Modificar campos de texto
* Llenar datos faltantes
* Estandarizar catálogos
* Eliminar datos fuera de rango
* Generar columnas con datos derivados
* Guardar datos

## 4.a Tipos de datos

Cómo vimos anteriormente hay varios tipos de datos como: texto, númerico, fecha, etc.
De manera automática pandas infiere el tipo de datos al leer el archivo, de una manera muy similar a como Excel interpreta si un dato es texto o numérico.

Los tipos de datos de pandas son:
* `object`: texto o mezcla de varios tipos de datos, este es el tipo de dato por defecto
* `category`: información categórica, similar a Factors en R
* `int64`: números enteros
* `float64`: números con punto decimal
* `bool`: valores verdadero o falso
* `datetime64`: fechas
* `timedelta[ns]`: diferencias de tiempo

Podemos ver los tipos de datos inferidos usando la opción `.dtypes`.

In [2]:
df.dtypes

Estado_origen                   object
Municipio_origen                object
Panteón_origen                  object
Estatus_FC                      object
Fecha_inhumación        datetime64[ns]
Fecha_defunción                 object
Fecha_exhumación        datetime64[ns]
Restos_tipo                     object
Sexo                            object
Edad                            object
Conocido_Desconocido            object
Primer apellido                 object
Segundo Apellido                object
Nombre(s)                       object
Nombre completo                 object
Institución_origen              object
Rdoc                            object
Marca_temporal          datetime64[ns]
Datos alternativos             float64
dtype: object

Dependiendo del tipo de dato se pueden hacer diferentes operaciones y hay tipos de limpieza específica.

Cómo vimos en la definición de datos ordenados o tidy data idealmente cada tabla tendrá solo datos de un tipo. Esto es para facilitar las operaciones. Si esto no es posible es importante que una columna o variable tenga datos de un solo tipo, ya que esto puede generar errores.

Por ejemplo, si tratamos de obtener el promedio de las edades nos genera un error de `ValueError`, ya que hay valores de texto `str` y número `int` mezclados, además de datos faltantes.

In [3]:
df['Edad'].unique()

array([55, nan, '19 semanas', '12 semanas', 31, 18, 39, 35, 74, 26,
       '36 semanas', '15 semanas', 83, 89, 64, 78, 73, 81, 75, 60, 63, 27,
       45, 40, 38, 25, 90, 42, 77, 70, 47, 32, '20 semanas', 76, 84, 87,
       66, 68, 94, 71, 92, 49, 69, 50, 80, 28, 37, 30, 20, '<17',
       '26 semanas', 34, 65, 86, 67, 58, 52, 85, '21 semanas',
       '17.6 semanas', '9 semanas', '33 semanas', 46, 22, '26.8 semanas',
       '22 semanas', 54, 33, 48, 53, 44, 57, 72, 41, 79, '29 semanas', 51,
       82, '67-82', 88, 36, 96, 62, 91, '13 semanas', '5 semanas',
       '11 semanas', '1 mes', '25 semanas', '26-32 semanas', '14 semanas',
       15, '40 semanas', '24 semanas', 24, 29, 13, 6, '23 semanas', 101,
       93, 59, 19, '13.7 semanas', 61, 97, '86-87', 95, 17, 16, 23, 21,
       '18 semanas', 56, '30 semanas', 11, 5, '30-40', 1], dtype=object)

Existen varias formas de fijar el tipo de datos, una de ellas es usar la opción `dtype` al leer el dataframe. Esta opción puede recibir un solo tipo de datos o un diccionarió de tipos de datos.

Por ejemplo, podemos leer toda la tabla cómo texto usando `dtype='str'`, lo cual obligará a ciertos tipos de datos como fechas o números a aparecer cómo texto. Hacer esto es una buena opción con los profiles iniciales cuando hay columnas que no se leen correctamente.

También se puede pasar un diccionario marcando el tipo de datos de cada columna específicamente. Sin embargo, si los datos no están limpios se pueden generar errores de lectura. Esta opción es ideal ya que los datos están limpios

In [4]:
df = pd.read_excel(file_mfc, sheet_name="MFC", nrows=1000, index_col='ID', dtype='str' )
df.head()

,Estado_origen,Municipio_origen,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,,,,
XX-P001,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,NaN,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23 00:00:00,NaN
XX-P002,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P003,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,NaN,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P004,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P005,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN


Los tipos de datos aparecen cómo object

In [5]:
df.dtypes

Estado_origen           object
Municipio_origen        object
Panteón_origen          object
Estatus_FC              object
Fecha_inhumación        object
Fecha_defunción         object
Fecha_exhumación        object
Restos_tipo             object
Sexo                    object
Edad                    object
Conocido_Desconocido    object
Primer apellido         object
Segundo Apellido        object
Nombre(s)               object
Nombre completo         object
Institución_origen      object
Rdoc                    object
Marca_temporal          object
Datos alternativos      object
dtype: object

Los números de `Edad` aparecen entre comillas ya que son texto.

In [6]:
df['Edad'].unique()

array(['55', nan, '19 semanas', '12 semanas', '31', '18', '39', '35',
       '74', '26', '36 semanas', '15 semanas', '83', '89', '64', '78',
       '73', '81', '75', '60', '63', '27', '45', '40', '38', '25', '90',
       '42', '77', '70', '47', '32', '20 semanas', '76', '84', '87', '66',
       '68', '94', '71', '92', '49', '69', '50', '80', '28', '37', '30',
       '20', '<17', '26 semanas', '34', '65', '86', '67', '58', '52',
       '85', '21 semanas', '17.6 semanas', '9 semanas', '33 semanas',
       '46', '22', '26.8 semanas', '22 semanas', '54', '33', '48', '53',
       '44', '57', '72', '41', '79', '29 semanas', '51', '82', '67-82',
       '88', '36', '96', '62', '91', '13 semanas', '5 semanas',
       '11 semanas', '1 mes', '25 semanas', '26-32 semanas', '14 semanas',
       '15', '40 semanas', '24 semanas', '24', '29', '13', '6',
       '23 semanas', '101', '93', '59', '19', '13.7 semanas', '61', '97',
       '86-87', '95', '17', '16', '23', '21', '18 semanas', '56',
       '30

Otra opción es convertir el tipo de dato columna por columna usando comandos como `.as_type` `.to_numeric` o `.to_datetime`.

![Función as_type](./images/pandas_astype.png)

Explicaremos esto con más detalle adelante.

## 4.b Eliminación de observaciones y variables

Es posible que no todos los datos contenidos en el conjunto de datos sean de interes. Eliminarlos facilita el análisis y eficientiza el uso de memoria.

Podemos eliminar filas o columnas si:
* Baja variabilidad, por ejemplo una columna donde todos los valores son iguales
* Información repetida, por ejemplo filas idénticas
* Existe una gran cantidad de datos faltantes
* Información no relevante para el análisis.

Mucha de esta información se puede obtener del diccionario de datos y el profile.

Por ejemplo, en el subconjunto que estamos usando las columnas 'Estado_origen', 'Municipio_origen' y 'Rdoc' tienen un solo valor, por lo que las eliminaremos

Podemos quitar las columnas usando el comando `.drop(axis=1)`. Si lo que queremos es quitar filas, se puede hacer de una manera similar, poniendo los nombres de las filas entre corchetes y usando el paramentro `axis=0`.

Para guardar estas modificaciones debemos de actualizar el dataframe

In [7]:
col_drop = ['Estado_origen', 'Municipio_origen', 'Rdoc']
df = df.drop( col_drop, axis=1)
df.head()

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,
XX-P001,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,NaN,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,2020-03-23 00:00:00,NaN
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P003,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,NaN,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN


Para determinar filas duplicadas usamos el comando `duplicated()`.

In [8]:
df[ df.duplicated() ]

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,
XX-P010,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P011,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P016,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-23 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P032,Panteón Civil de Dolores,Inhumación,2019-01-19 00:00:00,2018-12-04 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P041,Panteón Civil de Dolores,Inhumación,2019-01-19 00:00:00,2018-12-09 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P043,Panteón Civil de Dolores,Inhumación,2019-01-19 00:00:00,2018-12-09 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00,NaN
XX-P065,Panteón Civil de Dolores,Inhumación,2019-01-12 00:00:00,2018-12-01 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-24 00:00:00,NaN
XX-P066,Panteón Civil de Dolores,Inhumación,2019-01-12 00:00:00,2018-12-01 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-24 00:00:00,NaN
XX-P151,Panteón Civil de Dolores,Inhumación,2018-11-24 00:00:00,2018-10-08 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-24 00:00:00,NaN


Podemos ver que hay un número de filas que tienen datos duplicados, es decir, que todos los datos se parecen. 
En este caso todos son `Desconocidos` y tienen identificadores distintos, por lo que lo mas probable es que no sean duplicados de verdad, sino un caso de registros indistinguibles por la falta de información.

En caso de querer quitarlos se pueden quitar usando el comando `drop_duplicates()`, el cual incluye opciones para determinar cual de los duplicados conservar.


## 4.c Valores faltantes

Un valor faltante es un valor que falta o está incompleto en un conjunto de datos. A menudo, los valores faltantes se deben a errores de entrada de datos, fallas en la recolección de datos o problemas técnicos. Los valores faltantes pueden aparecer como espacios en blanco, ceros, letras u otros caracteres. La presencia de valores faltantes en un conjunto de datos puede afectar la precisión y la calidad del análisis de los datos, por lo que es importante identificarlos y manejarlos adecuadamente durante la limpieza y el preprocesamiento de los datos.

Otra razón para quitar datos es por que hay una gran cantidad de faltantes. El comando `dropna()` sirve para quitar estas filas (`axis=0`) o columnas (`axis=1`).
Existen varias opciones:

* `how='any'` quita las filas o columnas donde hay al menos un nan, este parametró es el default
* `how='all'` quita las filas o columnas donde todos son nan
* `thresh=numero` quita las filas o columnas con más de cierto número de nans, esto es útil para detectar automáticamente columnas donde faltan muchos datos

Quitemos las columnas donde todos los valores son nan, en este caso 'Datos alternativos':

In [9]:
df = df.dropna( axis=1, how='all' )
df.head()

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,,
XX-P001,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,NaN,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,2020-03-23 00:00:00
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P003,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,NaN,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00


Otra situación es cuando la columna o fila solo tiene algunos faltantes, en este caso es necesario tomar decisiones dependiendo de la situación.

Por ejemplo, la columna de 'Fecha_exhumación' tiene 99.9% de datos faltantes. 
Esta columna es un caso interesante, ya que contiene información del unico caso de exhumación de nuestro subconjunto de datos. Podríamos quitarla dada la cantidad de datos faltantes, pero antes de eso es importante ver la fila asociada.

In [10]:
df[ df['Fecha_exhumación'].notna() ]

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,,
XX-P331,Panteón Civil de Dolores,Exhumación,2019-03-02 00:00:00,2019-01-18 00:00:00,2020-03-31 00:00:00,Cadáver,Femenino,NaN,Conocido,Lopez,Gonzalez,Samara Judith,Samara Judith Lopez Gonzalez,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-27 18:16:02


En este caso es importante considerar que tipo de análisis nos interesa hacer y que información es necesaria.

Por ejemplo, si nos interesa las exhumaciones debemos de conservar la columna. Si nos interesan solo los cadaveres presentes en la fosa común se recomienda quitar la columna y la fila, ya que el cadaver ha sido removido. Sin embargo, si nos interesan las inhumanciones se debé de conservar la fila aunque se quite la columna, ya que el registro es tanto de una inhumación como de una exhumación. 
En general, es importante revisar los datos y tener claras las preguntas para tomar decisiones sobre la limpieza y análisis.

En este caso quitaremos las columnas con demasiados nan's, por ejemplo, dejaremos aquellas que tengan al menos 50 valores validos, lo cual debe de quitar 'Fecha_exhumación'

In [11]:
df = df.dropna(thresh=50, axis=1)
df.head()

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P001,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,2020-03-23 00:00:00
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P003,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00


Para determinar si debemos de quitar filas por nan en primer lugar hay que análizar la cantidad de faltantes y su distribución.

**Nota** Explicación del comando
```
df.isna()         <-- determinar si las celdas son na
  .sum(axis=1)    <-- sumar los faltantes por fila
  .value_counts() <-- contar cuantas veces aparece cada número
  .sort_index()   <-- ordenar por el indice, que en este caso es el número de faltantes
```


In [12]:
df.isna().sum(axis=1).value_counts().sort_index()

0    395
1    130
2     26
3    164
4    285
dtype: int64

Cómo podemos ver hay 285 filas donde faltan 4 valores, lo cual es poco menos de la mitad. Revisemos estas filas.

In [13]:
df[  df.isna().sum(axis=1)>=4  ]

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P006,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-16 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P008,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2019-12-17 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P966,Panteón Civil de Dolores,Inhumación,2018-07-14 00:00:00,2018-06-16 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-04-06 15:10:58.895000
XX-P967,Panteón Civil de Dolores,Inhumación,2018-07-14 00:00:00,2017-08-08 00:00:00,Cadáver,Femenino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-04-06 15:14:11.685000
XX-P974,Panteón Civil de Dolores,Inhumación,2018-07-14 00:00:00,S/D,Restos humanos,Indeterminado,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-04-06 18:51:06.834000


In [14]:
df[ df.isna().sum(axis=1)>=4 ]

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P006,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-16 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
XX-P008,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2019-12-17 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-03-23 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P966,Panteón Civil de Dolores,Inhumación,2018-07-14 00:00:00,2018-06-16 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-04-06 15:10:58.895000
XX-P967,Panteón Civil de Dolores,Inhumación,2018-07-14 00:00:00,2017-08-08 00:00:00,Cadáver,Femenino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-04-06 15:14:11.685000
XX-P974,Panteón Civil de Dolores,Inhumación,2018-07-14 00:00:00,S/D,Restos humanos,Indeterminado,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,2020-04-06 18:51:06.834000


En este caso son desconocidos donde no se sabe la edad, por lo que no es necesario quitarlas.

### Valores especiales

Otra opción es llenar los faltantes con un valor específico o especial. Por ejemplo, veamos la columna `Fecha_defunción`.
En esta columna hay varios tipos de valores faltantes, los cuales son representados no por `nan`, sino por 'S/D' y '??'

Esta es una forma de representar incertidumbre en los datos. Los datos faltantes pueden provenir de varias fuentes. Por ejemplo, en este caso el MFC incluye transcripciones de los registros de las fosas comunes del país. Es diferente que un dato no este en la fuente original a que no haya sido capturado al MFC. 

Existen varias formas de tratar estos datos faltantes, por ejemplo en este caso no se tiene la hora de defunción, por lo que por convención se representa la hora cómo 00:00:00, esto no significa que la persona haya muerto a esa hora, sino que es un valor default. El llenar los datos con valores default puede sesgar la distribución de los datos.

In [15]:
df['Fecha_defunción'].unique() [0:25]

array(['2018-03-04 00:00:00', '2018-12-12 00:00:00',
       '2018-12-13 00:00:00', '2018-12-14 00:00:00',
       '2018-12-18 00:00:00', '2018-12-16 00:00:00',
       '2019-12-17 00:00:00', '2018-12-30 00:00:00',
       '2018-12-21 00:00:00', '2018-12-19 00:00:00',
       '2018-12-23 00:00:00', '2018-12-27 00:00:00',
       '2019-12-28 00:00:00', '2018-12-29 00:00:00',
       '2019-12-31 00:00:00', 'S/D', '2018-12-03 00:00:00', '??/12/2018',
       '03/12/????', '2018-12-04 00:00:00', '2018-12-05 00:00:00',
       '2018-12-02 00:00:00', '2018-12-07 00:00:00',
       '2018-12-08 00:00:00', '2018-12-09 00:00:00'], dtype=object)

Otro ejemplo es lo que sucede con los nombres. En este caso tenemos varios valores especiales, ya que hay nombres de desconocidos que se representan con un espacio '  ', con `nan` o con el texto 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  '. Cada uno de estos representa diferentes tipos de falta de información y amerita un tratamiento diferente.

In [16]:
df['Nombre completo'].value_counts(dropna=False)[0:10].index

Index(['  ',
       'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
       'Javier  ', 'Miguel  ', 'Ricardo Antonio Márquez Cruz',
       'Andrés Aguilar Santana', 'Armando García Cortez', 'Leopoldo  ',
       'Macedonio Navarro García', 'Josefina De la Fuente Barajas'],
      dtype='object')

In [17]:
df['Nombre(s)'].value_counts(dropna=False)[0:10].index

Index([                                                                                                                               nan,
       'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
                                                                                                                                  'Jesús',
                                                                                                                                'Ricardo',
                                                                                                                                   'José',
                                                                                                                              'José Luis',
                                                                                                                                'Antonio',
                           

En el caso de datos númericos se puede sustituir con 'nan', cero, el promedio de los datos o un dato aleatorio de la población (un proceso similar a bootstrapping). Estas últimas opciones estan mas alla del alcance de este tutorial.

## 4.d Limpieza de texto

En este conjunto las columnas de texto son:
* 'Primer apellido'
* 'Segundo Apellido'
* 'Nombre(s)'
* 'Nombre completo'
* 'Institución_origen'
* 'Rdoc'

In [18]:
col_str = ['Primer apellido', 'Segundo Apellido', 'Nombre(s)',  
           'Nombre completo', 'Institución_origen']

# Este comando obtiene para cada columna de interes los cinco valores mas comunes
# y los despliega para poder revisar los datos
for col in col_str:
    print(col)
    display( df[col].value_counts(dropna=False).index.tolist()[0:5] )

Primer apellido


[nan,
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Hernández',
 'García',
 'Sánchez']

Segundo Apellido


[nan,
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Hernández',
 'González',
 'López']

Nombre(s)


[nan,
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Jesús',
 'Ricardo',
 'José']

Nombre completo


['  ',
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Javier  ',
 'Miguel  ',
 'Ricardo Antonio Márquez Cruz']

Institución_origen


['INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPERIOR DE JUSTICIA DE LA CIUDAD DE MÉXICO',
 'UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO - FACULTAD DE MEDICINA',
 'INSTITUTO POLITÉCNICO NACIONAL - ESCUELA NACIONAL DE MEDICINA Y HOMEOPATÍA',
 'CENTRO CULTURAL UNIVERSITARIO JUSTO SIERRA',
 'UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA']

Dentro de estos datos tenemos varios tipos de valores que nos representan distintos tipos de datos faltantes.

Estos son:
* `nan`
* `'  '`
* 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  '

Generalmente, podemos sustituir estos valores por `nan`, por un string vacio `''`o por un string especial cómo 'S/D' o 'Confidencial'.

En este caso vamos a sustituir por un string vacio: `''`

En primer lugar vamos a sustituir todos los `nan` usando el comando `.fillna()`.

In [19]:
# esta operación solo se aplica y guarda sobre las columnas de texto
df[col_str] = df[col_str].fillna('')

for col in col_str:
    print(col)
    display( df[col].value_counts(dropna=False).index.tolist()[0:5] )

Primer apellido


['',
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Hernández',
 'García',
 'Sánchez']

Segundo Apellido


['',
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Hernández',
 'González',
 'López']

Nombre(s)


['',
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Jesús',
 'Ricardo',
 'José']

Nombre completo


['  ',
 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ',
 'Javier  ',
 'Miguel  ',
 'Ricardo Antonio Márquez Cruz']

Institución_origen


['INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPERIOR DE JUSTICIA DE LA CIUDAD DE MÉXICO',
 'UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO - FACULTAD DE MEDICINA',
 'INSTITUTO POLITÉCNICO NACIONAL - ESCUELA NACIONAL DE MEDICINA Y HOMEOPATÍA',
 'CENTRO CULTURAL UNIVERSITARIO JUSTO SIERRA',
 'UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA']

Ahora, vamos a remplazar los demas valores especiales:

* `'  '` --> `''`
* 'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ' --> 'Confidencial'

En este caso crearemos un diccionario de remplazos y usaremos la función `.replace()`

In [20]:
replace_str = {'  ':'',
               'Nombre de particular que se encuentra con vida, se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.  ':'Confidencial'
              }

# remplazamos con un for loop columna por columna, en lugar de usar selección
# esto es solo por ejemplo

for col in col_str:
    print('Remplazando columna:', col)
    df[col] = df[col].replace(replace_str)
    # veamos los cinco valores mas comunes de la columna despúes de la sustitución
    display( df[col].value_counts(dropna=False).index.tolist()[0:5] )

Remplazando columna: Primer apellido


['', 'Confidencial', 'Hernández', 'García', 'Sánchez']

Remplazando columna: Segundo Apellido


['', 'Confidencial', 'Hernández', 'González', 'López']

Remplazando columna: Nombre(s)


['', 'Confidencial', 'Jesús', 'Ricardo', 'José']

Remplazando columna: Nombre completo


['', 'Confidencial', 'Javier  ', 'Miguel  ', 'Ricardo Antonio Márquez Cruz']

Remplazando columna: Institución_origen


['INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPERIOR DE JUSTICIA DE LA CIUDAD DE MÉXICO',
 'UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO - FACULTAD DE MEDICINA',
 'INSTITUTO POLITÉCNICO NACIONAL - ESCUELA NACIONAL DE MEDICINA Y HOMEOPATÍA',
 'CENTRO CULTURAL UNIVERSITARIO JUSTO SIERRA',
 'UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA']

Dependiendo del tipo de dato se pueden hacer diferentes operaciones. Por ejemplo, se puede pasar a mayusculas un texto pero no un número, se puede obtener el año de una fecha pero no de un texto

Por ejemplo, pensemos que queremos que las instituciones de origen no esten todas en mayusculas, sino solo la primera letra de la palabra.

Para hacer esto seleccionaremos la columna de interes, y luego aplicaremos la función `.str.title()`. 

**Nota**: existen varias funciones para mayusculas y minusculas como `.upper()`, `.lower()`, `.capitalize()` y `.title()`

In [21]:
df['Institución_origen'].str.title()

ID
XX-P001           Universidad Westhill - Facultad De Medicina
XX-P002     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P003     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P004     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P005     Instituto De Ciencias Forenses - Tribunal Supe...
                                  ...                        
XX-P996     Universidad Nacional Autónoma De México - Facu...
XX-P997     Universidad Nacional Autónoma De México - Facu...
XX-P998     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P999     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P1000    Instituto De Ciencias Forenses - Tribunal Supe...
Name: Institución_origen, Length: 1000, dtype: object

Existen funciones que no se encuentran en pandas pero que pueden ser utiles. Por ejemplo, para quitar los acentos es posible con la función `unidecode()`, la cual es parte de la biblioteca `unidecode`.

In [22]:
from unidecode import unidecode
unidecode('México')

'Mexico'

Podemos aplicar esta función a toda la columna, en este caso usaremos apply, ya que no es una función default de pandas

In [23]:
df['Institución_origen'].apply( unidecode )

ID
XX-P001           UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA
XX-P002     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P003     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P004     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P005     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
                                  ...                        
XX-P996     UNIVERSIDAD NACIONAL AUTONOMA DE MEXICO - FACU...
XX-P997     UNIVERSIDAD NACIONAL AUTONOMA DE MEXICO - FACU...
XX-P998     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P999     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P1000    INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
Name: Institución_origen, Length: 1000, dtype: object

Además, podemos aplicar varias funciones juntas.

In [24]:
df['Institución_origen'].str.title().apply(unidecode)

ID
XX-P001           Universidad Westhill - Facultad De Medicina
XX-P002     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P003     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P004     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P005     Instituto De Ciencias Forenses - Tribunal Supe...
                                  ...                        
XX-P996     Universidad Nacional Autonoma De Mexico - Facu...
XX-P997     Universidad Nacional Autonoma De Mexico - Facu...
XX-P998     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P999     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P1000    Instituto De Ciencias Forenses - Tribunal Supe...
Name: Institución_origen, Length: 1000, dtype: object

Estas modificaciones son temporales. Para guardar los resultados es necesario guardar la columna o serie generada. Esto puede ser en una nueva columna o en la misma columna

In [25]:
df['Institución_origen'] = df['Institución_origen'].str.title().apply(unidecode)
df.tail()

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,S/D,Cadáver,Masculino,42,Conocido,Flores,Rosette,Sergio,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,2020-04-06 21:19:52.451000
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2017-10-27 00:00:00,Cadáver,Masculino,64,Conocido,Juárez,Chávez,Juan Carlos,Juan Carlos Juárez Chávez,Universidad Nacional Autonoma De Mexico - Facu...,2020-04-06 21:22:53.304000
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-08-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:30:12.169000
XX-P999,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-05-22 00:00:00,Cadáver,Femenino,1,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:32:30.839000
XX-P1000,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-05-29 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:35:23.157000


EL aplicar la función `unidecode()` a la columna puede genera un error si hay nans `AttributeError: 'float' object has no attribute 'encode'`. 
Esto se debe a que la columna incluye valores vacios, los cuales son de tipo `float`, por lo tanto la función `unidecode` falla, ya que los números no tienen acentos.

Dejo aqui una posible solución. Esta solución utiliza funciones lambda, las cuales son un tema avanzado más allá de este tutorial.

In [26]:
df['Nombre(s)'].apply( lambda s: unidecode(s) if type(s)==str  else s )

ID
XX-P001                
XX-P002                
XX-P003         Alberto
XX-P004                
XX-P005                
               ...     
XX-P996          Sergio
XX-P997     Juan Carlos
XX-P998                
XX-P999                
XX-P1000               
Name: Nombre(s), Length: 1000, dtype: object

Otra posible limpieza es quitar espacios al principio y final del texto, o cambiar tabuladores, saltos de página o múltiples espacios por uno solo. 
Este tipo de ocurrencias son dificiles de ver en el texto como humanos, sin embargo un espacio adicional al final de una palabra es suficiente para volverla un string diferente para la computadora. Por ejemplo, veamos este texto que incluye múltiples espaciós, tabuladores y saltos de linea.

In [27]:
texto = "  Emanuel\tJurado     Sánchez \n "
print(texto)

  Emanuel	Jurado     Sánchez 
 


La función `.strip()` quita los espacios adicionales del principio y final de un string.

In [28]:
print(texto.strip())

Emanuel	Jurado     Sánchez


Otra opción es combinar la función `.split()` y `.join()`.
La función `.split()` separa un texto en una lista, por default hace esto en los caracteres de espacio, pero se le pueden dar instrucciones de separar en caracteres específicos, por ejemplo en una coma `,`.
La función  `str.join()` une una lista con un caracter específico para generar un solo string.

En este caso estamos separando el texto en los espacios, para convertirlo en una lista de palabras. Despúes, esta lista se une con espacios.


In [29]:
print(   ' '.join( texto.split() )   )

Emanuel Jurado Sánchez


Es común que una limpieza requiera varios pasos. En ese caso podemos generar una función que incluya todos los pasos.

In [30]:
def limpiar_texto(s):
    # checar el tipo
    if type(s)==str: # esto solo se ejecutará si es un string
        ### aqui agrega tus pasos de limpieza 
        s = unidecode(s) #quitar acento
        s = ' '.join( s.split() ) #quitar espacios extra
        s = s.title() #primera letra en mayuscula
    else: # esto se ejecutará si NO es un string
        s = s #devolveremos la entrada sin modificaciones, pero puedes cambiar esto
    return s
    
limpiar_texto(texto)

'Emanuel Jurado Sanchez'

Ahora aplicaremos la función en las columnas de interes.

In [31]:
for col in col_str:
    print('Limpieza de texto:', col)
    df[col] = df[col].apply( limpiar_texto )
df.tail()

Limpieza de texto: Primer apellido
Limpieza de texto: Segundo Apellido
Limpieza de texto: Nombre(s)
Limpieza de texto: Nombre completo
Limpieza de texto: Institución_origen


,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,S/D,Cadáver,Masculino,42,Conocido,Flores,Rosette,Sergio,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,2020-04-06 21:19:52.451000
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2017-10-27 00:00:00,Cadáver,Masculino,64,Conocido,Juarez,Chavez,Juan Carlos,Juan Carlos Juarez Chavez,Universidad Nacional Autonoma De Mexico - Facu...,2020-04-06 21:22:53.304000
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-08-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:30:12.169000
XX-P999,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-05-22 00:00:00,Cadáver,Femenino,1,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:32:30.839000
XX-P1000,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-05-29 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:35:23.157000


## 4.e Limpieza de categóricos

En general se considera que un dato es categorico cuando hay pocas posibilidades definidas. Una aproximación rápida es que tenga menos de diez opciones de texto. Sin embargo esta no es una regla dura, por ejemplo el catálogo del INEGI de Estados, Municipios y Localidades tiene miles de opciones.


En este conjunto las columnas de texto son:
* 'Panteón_origen', 
* 'Estatus_FC', 
* 'Restos_tipo', 
* 'Sexo', 
* 'Conocido_Desconocido'

In [32]:
col_cat = ['Panteón_origen', 'Estatus_FC', 'Restos_tipo', 'Sexo', 'Conocido_Desconocido']

# este es el mismo comando que usamos para ver col_str, pero cambiamos la lista que se usará
for col in col_cat:
    print(col)
    display( df[col].value_counts(dropna=False).index.tolist() )

Panteón_origen


['Panteón Civil de Dolores']

Estatus_FC


['Inhumación', 'Exhumación']

Restos_tipo


['Cadáver',
 'Restos cremados',
 'Miembros',
 'Feto',
 'Restos humanos',
 'Restos óseos']

Sexo


['Masculino', 'Femenino', 'Indeterminado']

Conocido_Desconocido


['Conocido', 'Desconocido']

Podemos convertir una columna de tipo `object` a `category` con la función `.astype("category")`. 

En este ejemplo caso reescribiremos la columna para poder realizar varias operaciones.

In [33]:
for col in col_cat:
    df[col] = df[col].astype("category")

Podemos ver que el tipo a cambiado

In [34]:
df.dtypes

Panteón_origen          category
Estatus_FC              category
Fecha_inhumación          object
Fecha_defunción           object
Restos_tipo             category
Sexo                    category
Edad                      object
Conocido_Desconocido    category
Primer apellido           object
Segundo Apellido          object
Nombre(s)                 object
Nombre completo           object
Institución_origen        object
Marca_temporal            object
dtype: object

Una ventaja de usar `category` es que permite definir un orden diferente al alfabético o numérico. Por ejemplo, definamos un orden especial para el tipo de restos. Por default las categorías se ordenan alfabeticamente, pero esto se puede modificar.

In [35]:
from pandas.api.types import CategoricalDtype

orden_restos = ['Cadáver', 'Restos humanos', 'Restos cremados', 'Restos óseos', 'Feto', 'Miembros']
orden_restos = CategoricalDtype(categories=orden_restos, ordered=True)
df['Restos_tipo'] = df['Restos_tipo'].astype( orden_restos )
df['Restos_tipo'].tail()

ID
XX-P996     Cadáver
XX-P997     Cadáver
XX-P998     Cadáver
XX-P999     Cadáver
XX-P1000    Cadáver
Name: Restos_tipo, dtype: category
Categories (6, object): ['Cadáver' < 'Restos humanos' < 'Restos cremados' < 'Restos óseos' < 'Feto' < 'Miembros']

Ahora si ordenamos el dataframe se usará el orden especial qué hemos definido

In [36]:
df.sort_values(by='Restos_tipo')

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P1000,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-05-29 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-06 21:35:23.157000
XX-P516,Panteón Civil de Dolores,Inhumación,2019-03-16 00:00:00,2019-02-04 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-03-30 13:15:04
XX-P517,Panteón Civil de Dolores,Inhumación,2019-05-16 00:00:00,2018-07-04 00:00:00,Cadáver,Masculino,70,Conocido,,,German,German,Universidad Anahuac - Facultad De Ciencias De ...,2020-03-30 13:15:35
XX-P518,Panteón Civil de Dolores,Inhumación,2019-04-06 00:00:00,2019-04-03 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-03-30 13:16:51
XX-P855,Panteón Civil de Dolores,Inhumación,2018-09-29 00:00:00,2018-08-19 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,2020-04-03 11:53:02.583000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P792,Panteón Civil de Dolores,Inhumación,2017-10-07 00:00:00,S/D,Miembros,Masculino,23,Conocido,Confidencial,Confidencial,Confidencial,Confidencial,Instituto De Ciencias Forenses - Tribunal Supe...,2020-01-04 16:40:40
XX-P683,Panteón Civil de Dolores,Inhumación,2017-11-24 00:00:00,S/D,Miembros,Masculino,NaN,Conocido,Confidencial,Confidencial,Confidencial,Confidencial,The American British Cowdray Medical Center La.P.,2020-02-04 11:42:51
XX-P682,Panteón Civil de Dolores,Inhumación,2017-11-24 00:00:00,S/D,Miembros,Femenino,74,Conocido,Confidencial,Confidencial,Confidencial,Confidencial,The American British Cowdray Medical Center La.P.,2020-01-04 17:27:36


### Creando columnas categóricas

Cuando se generan nuevas variables de datos a partir de las existentes se le llama "creación de características" o "ingeniería de características" (en inglés, feature engineering). Es una técnica comúnmente utilizada en el análisis de datos para mejorar la precisión de los modelos de machine learning o para encontrar patrones y tendencias en los datos. La idea es generar nuevas variables a partir de las ya existentes que puedan ser más relevantes o informativas para el problema en cuestión.

Veamos las instituciones de origen:

In [37]:
df['Institución_origen'].value_counts()

Instituto De Ciencias Forenses - Tribunal Superior De Justicia De La Ciudad De Mexico    533
Universidad Nacional Autonoma De Mexico - Facultad De Medicina                           331
Instituto Politecnico Nacional - Escuela Nacional De Medicina Y Homeopatia                58
Centro Cultural Universitario Justo Sierra                                                16
Universidad Westhill - Facultad De Medicina                                               15
Universidad Anahuac - Facultad De Ciencias De La Salud                                    13
The American British Cowdray Medical Center La.P.                                          9
Secretaria De La Defensa Nacional - Escuela Militar De Medicina                            7
Universidad Popular Autonoma Del Estado De Puebla                                          7
Universidad Tominaga Nakamoto S.C. Escuela De Medicina Ciencias Basicas                    4
Escuela De Medicina Saint Luke                                        

Podemos ver que los cadaveres provienen de Instituciones Judiciales como en INCIFO y la PGR, de escuelas públicas y privadas. Entonces, generemos una nueva columna con el tipo de institución de origen.

Para esto primero crearemos un diccionario que nos permita mapear las instituciones a su tipo.

In [38]:
cat_tipo_inst = {
        'Instituto De Ciencias Forenses - Tribunal Superior De Justicia De La Ciudad De Mexico': 'Institución judicial',
        'Procuraduria General De La Republica': 'Institución judicial',
        'Universidad Nacional Autonoma De Mexico - Facultad De Medicina': 'Escuela pública',
        'Instituto Politecnico Nacional - Escuela Nacional De Medicina Y Homeopatia': 'Escuela pública',
        'Centro Cultural Universitario Justo Sierra': 'Escuela privada',
        'Universidad Westhill - Facultad De Medicina': 'Escuela privada',
        'Universidad Anahuac - Facultad De Ciencias De La Salud': 'Escuela privada',
        'The American British Cowdray Medical Center La.P.': 'Escuela privada',
        'Secretaria De La Defensa Nacional - Escuela Militar De Medicina': 'Escuela privada',
        'Universidad Popular Autonoma Del Estado De Puebla': 'Escuela privada',
        'Universidad Tominaga Nakamoto S.C. Escuela De Medicina Ciencias Basicas': 'Escuela privada',
        'Escuela De Medicina Saint Luke': 'Escuela privada',
        'Institucion De Asistencia Privada - Escuela Libre De Homeopatia De Mexico': 'Escuela privada',
        }

A continuación usaremos map para generar una nueva columna usando el mapeo.

Guardaremos el resultado de esta operación en una columna nueva llamada 'Tipo_institución_origen'

**Nota** `.replace()` y `.map()` son funciones que toman un diccionario y lo utilizan para remplazar los valores. Sin embargo, `.replace()` solo sustituye los valores en el diccionario y si algo no se encuentra el valor en el dict deja la celda sin modificar. Por otro lado `.map()` cambia todos los valores en el diccionario y si no se encuentran el el dict lo sustituye por `nan`, por lo cual es útil para filtrar valores no validos.


In [39]:
df['Tipo_institución_origen'] = df['Institución_origen'].map(cat_tipo_inst)
df['Tipo_institución_origen'].value_counts(dropna=False)

Institución judicial    535
Escuela pública         389
Escuela privada          76
Name: Tipo_institución_origen, dtype: int64

Ahora, hay que volver este dato de tipo categórico

In [40]:
df['Tipo_institución_origen'] = df['Tipo_institución_origen'].astype("category")
df['Tipo_institución_origen'].head()

ID
XX-P001         Escuela privada
XX-P002    Institución judicial
XX-P003    Institución judicial
XX-P004    Institución judicial
XX-P005    Institución judicial
Name: Tipo_institución_origen, dtype: category
Categories (3, object): ['Escuela privada', 'Escuela pública', 'Institución judicial']

### Wibblywobbly o catálogos por similitud

A veces es necesario estandarizar los datos antes de volverlos una categoría.

Por ejemplo, estos serie de datos esta basada en datos llenados a mano sobre el sexo de pacientes.


In [41]:
serie_sexo = pd.Series( [' Masculino', '-', '?', 'F', 'Femenina', 'Femenino', 'Femenino y Masculino', 'I?','Ilegible', 
                         'Indeterminable', 'Indeterminado', 'M', 'M F', 'MAsculino', 'Maculino', 'Masculino', 'Masculino.', 
                         'N', 'S/D', 'femenina', 'femenino', 'ilegible', 'indeterminado', 'masculino', 's/d', 'sin dato' ] ) 
serie_sexo

0                Masculino
1                        -
2                        ?
3                        F
4                 Femenina
5                 Femenino
6     Femenino y Masculino
7                       I?
8                 Ilegible
9           Indeterminable
10           Indeterminado
11                       M
12                     M F
13               MAsculino
14                Maculino
15               Masculino
16              Masculino.
17                       N
18                     S/D
19                femenina
20                femenino
21                ilegible
22           indeterminado
23               masculino
24                     s/d
25                sin dato
dtype: object

En primer lugar tenemos que determinar cuales opciones incluir en el catalogo. Además, es necesario tomar decisiones de registros dobles como 'M F'. 
En este caso nos gustaría reducir las opciones a 'Femenino', 'Masculino' e 'Indeterminado', donde 'Indeterminado' incluira todos los casos donde haya duda.

Para lograr la estandarización se generá un diccionario de equivalencias, el cual se usa con la función `.replace()` o `.map()`. 

Este diccionario se puede escribir a mano cómo se hizo arriba o inferir utilizando `wibblywobbly`. Esta biblioteca toma un catálogo y un conjunto de datos y regresa que tanto se parecen los textos.

In [42]:
import wibblywobbly as ww

cat_sexo = ['Femenino', 'Masculino', 'Indeterminado']
ww.map_list_to_catalog(serie_sexo, cat_sexo, reject_value='Indeterminado')

/home/esther/anaconda3/envs/CursoPandas2023/lib/python3.10/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,Data,Option1,Score1,Option2,Score2,Option3,Score3
0,masculino,Masculino,100,None,NaN,None,NaN
1,indeterminado,Indeterminado,100,None,NaN,None,NaN
2,femenino,Femenino,100,None,NaN,None,NaN
3,Masculino.,Masculino,100,None,NaN,None,NaN
4,Masculino,Masculino,100,None,NaN,None,NaN
5,MAsculino,Masculino,100,None,NaN,None,NaN
6,Indeterminado,Indeterminado,100,None,NaN,None,NaN
7,Femenino,Femenino,100,None,NaN,None,NaN
8,Masculino,Masculino,100,None,NaN,None,NaN
9,Maculino,Masculino,94,None,NaN,None,NaN


También es posible obtener directamente un diccionario de sustitución.

La opción `reject_value` permite establecer un valor default si no se encuentra un texto lo suficientemente similar en el catálogo.

In [43]:
replace_sexo = ww.map_list_to_catalog(serie_sexo, cat_sexo, output_format="dictionary", reject_value='Indeterminado')
replace_sexo

{'Indeterminable': 'Indeterminado',
 '?': 'Indeterminado',
 'F': 'Femenino',
 ' Masculino': 'Masculino',
 'S/D': 'Indeterminado',
 'Indeterminado': 'Indeterminado',
 'Masculino': 'Masculino',
 'ilegible': 'Indeterminado',
 'indeterminado': 'Indeterminado',
 'M F': 'Indeterminado',
 'Masculino.': 'Masculino',
 's/d': 'Indeterminado',
 'Maculino': 'Masculino',
 '-': 'Indeterminado',
 'MAsculino': 'Masculino',
 'femenino': 'Femenino',
 'Ilegible': 'Indeterminado',
 'N': 'Femenino',
 'masculino': 'Masculino',
 'Femenino y Masculino': 'Masculino',
 'Femenino': 'Femenino',
 'I?': 'Femenino',
 'M': 'Femenino',
 'femenina': 'Femenino',
 'sin dato': 'Indeterminado',
 'Femenina': 'Femenino'}

Este diccionario tiene errores, los cuales se pueden arreglar manualmente.

In [44]:
replace_sexo['M'] = 'Masculino'
replace_sexo['N'] = 'Indeterminado'
replace_sexo['I?'] = 'Indeterminado'
replace_sexo['Femenino y Masculino'] = 'Indeterminado'

replace_sexo

{'Indeterminable': 'Indeterminado',
 '?': 'Indeterminado',
 'F': 'Femenino',
 ' Masculino': 'Masculino',
 'S/D': 'Indeterminado',
 'Indeterminado': 'Indeterminado',
 'Masculino': 'Masculino',
 'ilegible': 'Indeterminado',
 'indeterminado': 'Indeterminado',
 'M F': 'Indeterminado',
 'Masculino.': 'Masculino',
 's/d': 'Indeterminado',
 'Maculino': 'Masculino',
 '-': 'Indeterminado',
 'MAsculino': 'Masculino',
 'femenino': 'Femenino',
 'Ilegible': 'Indeterminado',
 'N': 'Indeterminado',
 'masculino': 'Masculino',
 'Femenino y Masculino': 'Indeterminado',
 'Femenino': 'Femenino',
 'I?': 'Indeterminado',
 'M': 'Masculino',
 'femenina': 'Femenino',
 'sin dato': 'Indeterminado',
 'Femenina': 'Femenino'}

Ahora podemos remplazar, estandarizar y contar.

In [45]:
serie_sexo = serie_sexo.replace( replace_sexo )
serie_sexo

0         Masculino
1     Indeterminado
2     Indeterminado
3          Femenino
4          Femenino
5          Femenino
6     Indeterminado
7     Indeterminado
8     Indeterminado
9     Indeterminado
10    Indeterminado
11        Masculino
12    Indeterminado
13        Masculino
14        Masculino
15        Masculino
16        Masculino
17    Indeterminado
18    Indeterminado
19         Femenino
20         Femenino
21    Indeterminado
22    Indeterminado
23        Masculino
24    Indeterminado
25    Indeterminado
dtype: object

In [46]:
serie_sexo.value_counts(dropna=False)

Indeterminado    14
Masculino         7
Femenino          5
dtype: int64

Revisen la documentación en: https://github.com/mar-esther23/WibblyWobbly

## 4.f Limpieza de números

Existen dos tipos de datos númericos, enteros y flotantes. Los dos se comportan de forma muy similar.

En este conjunto de datos la columna `Edad` contiene numeros, sin embargo su tipo es `object` ya que hay edades como "18 semanas" que corresponden a fetos y neonatos.
Además, cuando cargamos específicamos que el tipo era `str`. 


In [47]:
df['Edad'].value_counts()

50               21
60               20
40               19
70               18
78               17
                 ..
25 semanas        1
26-32 semanas     1
15                1
40 semanas        1
1                 1
Name: Edad, Length: 118, dtype: int64

Por lo tanto, es necesario volver la edad a número. Esto se puede hacer con la función `.to_numeric()` o con `.astype()`.
Los datos que no se pueden convertir a número, por ejemplo '18 semanas', ya que generarán error. La función tiene varios parametros que nos pueden servir:
```
errors{‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
        If ‘raise’, then invalid parsing will raise an exception.
        If ‘coerce’, then invalid parsing will be set as NaN.
        If ‘ignore’, then invalid parsing will return the input.
```

Una primera aproximación es convertir obligar a que se convierta a número. Cómo los datos de semanas nos pueden ser utiles a futuro no vamos a rescribir la columna 'Edad', sino a generar una nueva columna 'Edad_int' con los resultados

In [48]:
df['Edad_int'] = pd.to_numeric(df['Edad'], errors="coerce")
df['Edad_int'].unique()

array([ 55.,  nan,  31.,  18.,  39.,  35.,  74.,  26.,  83.,  89.,  64.,
        78.,  73.,  81.,  75.,  60.,  63.,  27.,  45.,  40.,  38.,  25.,
        90.,  42.,  77.,  70.,  47.,  32.,  76.,  84.,  87.,  66.,  68.,
        94.,  71.,  92.,  49.,  69.,  50.,  80.,  28.,  37.,  30.,  20.,
        34.,  65.,  86.,  67.,  58.,  52.,  85.,  46.,  22.,  54.,  33.,
        48.,  53.,  44.,  57.,  72.,  41.,  79.,  51.,  82.,  88.,  36.,
        96.,  62.,  91.,  15.,  24.,  29.,  13.,   6., 101.,  93.,  59.,
        19.,  61.,  97.,  95.,  17.,  16.,  23.,  21.,  56.,  11.,   5.,
         1.])

Veamos las edades que no fueron convertidas por la operación:

In [49]:
df.loc[ (df['Edad'].notna()) & (df['Edad_int'].isna()), ['Restos_tipo','Edad','Edad_int'] ].drop_duplicates()

,Restos_tipo,Edad,Edad_int
ID,,,
XX-P022,Feto,19 semanas,NaN
XX-P023,Feto,12 semanas,NaN
XX-P069,Feto,36 semanas,NaN
XX-P071,Feto,15 semanas,NaN
XX-P115,Feto,20 semanas,NaN
XX-P167,Cadáver,<17,NaN
XX-P168,Feto,26 semanas,NaN
XX-P207,Feto,21 semanas,NaN
XX-P208,Feto,17.6 semanas,NaN


La mayor parte de las edades en semanas son de fetos, mientras que las excepciones corresponden a rango. Dada la cantidad de faltantes ignoraremos estos datos. Sin embargo, es buena práctica revisar el resultado de las conversiones y donde fallo.

Ahora, veamos la distribución de edades:

In [50]:
df['Edad_int'].describe()

count    540.000000
mean      57.733333
std       20.535846
min        1.000000
25%       40.750000
50%       60.000000
75%       74.250000
max      101.000000
Name: Edad_int, dtype: float64

Tenemos una persona de 101 años, esto podría ser un dato fuera de rango. 
En este caso vamos a poner un críterio para manejar problemas similares a futuro, si la persona tiene mas de cién años remplazaremos su edad por nan.
Para lograr esto seleccionaremos todas las celdas de personas de mas de cién años que están en la columna 'Edad_años' con _.loc[]_ y remplazaremos estos valores por NaN.
Esto debé de cambiar la edad máxima de nuestros datos.

In [51]:
from numpy import nan

df.loc[ df['Edad_int']>=100,'Edad_int' ] = nan
df['Edad_int'].max()

97.0

Veamos las estadísticas de la columna 'Edad_años' usando `.describe()`, esta función es una alternativa rápida a un profile.

In [52]:
df['Edad_int'].describe()

count    539.000000
mean      57.653061
std       20.469949
min        1.000000
25%       40.500000
50%       60.000000
75%       74.000000
max       97.000000
Name: Edad_int, dtype: float64

## 4.g Limpieza de fechas

Las fechas se encuentran en formato `datetime64`, lo cual incluye, fecha y hora.
Este formato sigue el patrón: `yyyy-mm-dd hh:mm:ss`

En este conjunto las columnas de texto son:
* 'Fecha_inhumación'
* 'Fecha_defunción'
* 'Marca_temporal'

In [53]:
col_date = ['Fecha_inhumación', 'Fecha_defunción', 'Marca_temporal']

for col in col_date:
    print(col)
    display( df[col].value_counts(dropna=False).index.tolist()[0:5] )

Fecha_inhumación


['2019-03-30 00:00:00',
 '2018-07-07 00:00:00',
 '2019-05-04 00:00:00',
 '2017-10-14 00:00:00',
 '2018-11-24 00:00:00']

Fecha_defunción


['S/D',
 '2018-06-16 00:00:00',
 '2018-12-18 00:00:00',
 '2019-02-14 00:00:00',
 '2019-02-08 00:00:00']

Marca_temporal


['2020-03-24 00:00:00',
 '2020-03-23 00:00:00',
 '2020-04-07 00:00:00',
 '2020-02-04 11:50:57',
 '2020-04-06 21:32:30.839000']

Convertiremos las fechas a datetime con `to_datetime()` y veremos en que casos fracasa la función.
Guardaremos el resultado en una columna con el sufijo `_date`.

**Nota:** Para entender esta función es muy importante ver en donde se agrega el modificador que indica una nueva columna

In [54]:
for col in col_date:
    print('Convirtiendo columna:' + col)
    df[col+'_date'] = pd.to_datetime(df[col], errors='coerce')
    display(df.loc[ (df[col].notna()) & (df[col+'_date'].isna()), [col, col+'_date'] ].drop_duplicates())

Convirtiendo columna:Fecha_inhumación


,Fecha_inhumación,Fecha_inhumación_date
ID,,


Convirtiendo columna:Fecha_defunción


,Fecha_defunción,Fecha_defunción_date
ID,,
XX-P022,S/D,NaT
XX-P029,??/12/2018,NaT
XX-P030,03/12/????,NaT


Convirtiendo columna:Marca_temporal


,Marca_temporal,Marca_temporal_date
ID,,


En este caso se puede ver que la conversión fallo solo en tres casos de la columna 'Fecha_defunción'. Cómo en estos casos falta información no es necesario hacer otra limpieza.

Usando este tipo de dato ahora es posible seleccionar por año, mes, día, hora y minuto usando los comandos:

* columna.dt.year
* columna.dt.month
* columna.dt.day
* columna.dt.hour
* columna.dt.minute
* columna.dt.dayofweek (numerico)
* columna.dt.weekday (nombre en ingles)
* columna.dt.dayofyear (numerico)
* columna.dt.weekofyear (numerico)

Agregaremos columnas para marcar el dia de la semana, del año y número de semana.

Es importante recordar que aplicaremos esto sobre las columnas procesadas con sufijo '_date'

In [55]:
for col in col_date:
    col = col+'_date' #usemos columna modificada
    df[col+'_diasemana'] = df[col].dt.weekday
    df[col+'_diaaño'] = df[col].dt.dayofyear
    df[col+'_semanaaño'] = df[col].dt.weekofyear
df

/tmp/ipykernel_15588/2670461742.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df[col+'_semanaaño'] = df[col].dt.weekofyear
/tmp/ipykernel_15588/2670461742.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df[col+'_semanaaño'] = df[col].dt.weekofyear
/tmp/ipykernel_15588/2670461742.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df[col+'_semanaaño'] = df[col].dt.weekofyear


,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,...,Marca_temporal_date,Fecha_inhumación_date_diasemana,Fecha_inhumación_date_diaaño,Fecha_inhumación_date_semanaaño,Fecha_defunción_date_diasemana,Fecha_defunción_date_diaaño,Fecha_defunción_date_semanaaño,Marca_temporal_date_diasemana,Marca_temporal_date_diaaño,Marca_temporal_date_semanaaño
ID,,,,,,,,,,,,,,,,,,,,,
XX-P001,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,Restos cremados,Masculino,55,Desconocido,,,...,2020-03-23 00:00:00.000,2,30,5,6.0,63.0,9.0,0,83,13
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-03-23 00:00:00.000,5,26,4,2.0,346.0,50.0,0,83,13
XX-P003,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,...,2020-03-23 00:00:00.000,5,26,4,3.0,347.0,50.0,0,83,13
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-03-23 00:00:00.000,5,26,4,4.0,348.0,50.0,0,83,13
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-03-23 00:00:00.000,5,26,4,1.0,352.0,51.0,0,83,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,S/D,Cadáver,Masculino,42,Conocido,Flores,Rosette,...,2020-04-06 21:19:52.451,5,244,35,NaN,NaN,NaN,0,97,15
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2017-10-27 00:00:00,Cadáver,Masculino,64,Conocido,Juarez,Chavez,...,2020-04-06 21:22:53.304,5,244,35,4.0,300.0,43.0,0,97,15
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-08-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-04-06 21:30:12.169,5,244,35,5.0,230.0,33.0,0,97,15


Al agregar columnas es importante tomar en cuenta el tipo de dato. Por ejemplo, veamos las columnas de día de la semana. Estas asignan un número al día de la semana con Lunes=0 y Domingo=6.

In [56]:
df['Marca_temporal_date_diasemana'].unique()

array([0, 1, 4, 5, 3, 6])

Editemos estas columnas para que tengan el día de la semana por su nombre y sean categóricos ordenados.

Lo primero es hacer un diccionario para sustituir los valores por el nombre del día de la semana.

In [57]:
dic_semana = {0:'Lunes', 1:'Martes', 2:'Miercoles', 3:'Jueves',
              4:'Viernes', 5:'Sábado', 6:'Domingo'}

col_weekday = ['Fecha_inhumación_date_diasemana', 'Fecha_defunción_date_diasemana',
               'Marca_temporal_date_diasemana']

for col in col_weekday:
    df[col] = df[col].replace(dic_semana)
df

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,...,Marca_temporal_date,Fecha_inhumación_date_diasemana,Fecha_inhumación_date_diaaño,Fecha_inhumación_date_semanaaño,Fecha_defunción_date_diasemana,Fecha_defunción_date_diaaño,Fecha_defunción_date_semanaaño,Marca_temporal_date_diasemana,Marca_temporal_date_diaaño,Marca_temporal_date_semanaaño
ID,,,,,,,,,,,,,,,,,,,,,
XX-P001,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,Restos cremados,Masculino,55,Desconocido,,,...,2020-03-23 00:00:00.000,Miercoles,30,5,Domingo,63.0,9.0,Lunes,83,13
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-03-23 00:00:00.000,Sábado,26,4,Miercoles,346.0,50.0,Lunes,83,13
XX-P003,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,...,2020-03-23 00:00:00.000,Sábado,26,4,Jueves,347.0,50.0,Lunes,83,13
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-03-23 00:00:00.000,Sábado,26,4,Viernes,348.0,50.0,Lunes,83,13
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-03-23 00:00:00.000,Sábado,26,4,Martes,352.0,51.0,Lunes,83,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,S/D,Cadáver,Masculino,42,Conocido,Flores,Rosette,...,2020-04-06 21:19:52.451,Sábado,244,35,NaN,NaN,NaN,Lunes,97,15
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2017-10-27 00:00:00,Cadáver,Masculino,64,Conocido,Juarez,Chavez,...,2020-04-06 21:22:53.304,Sábado,244,35,Viernes,300.0,43.0,Lunes,97,15
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01 00:00:00,2018-08-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,,,...,2020-04-06 21:30:12.169,Sábado,244,35,Sábado,230.0,33.0,Lunes,97,15


Ahora, generemos la categoría ordenada y transformemos las columnas a categórico.


In [58]:
orden_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves',
                'Viernes', 'Sábado', 'Domingo']
orden_semana = CategoricalDtype(categories=orden_semana, ordered=True)

for col in col_weekday:
    df[col] = df[col].astype( orden_semana )
df.dtypes

Panteón_origen                           category
Estatus_FC                               category
Fecha_inhumación                           object
Fecha_defunción                            object
Restos_tipo                              category
Sexo                                     category
Edad                                       object
Conocido_Desconocido                     category
Primer apellido                            object
Segundo Apellido                           object
Nombre(s)                                  object
Nombre completo                            object
Institución_origen                         object
Marca_temporal                             object
Tipo_institución_origen                  category
Edad_int                                  float64
Fecha_inhumación_date              datetime64[ns]
Fecha_defunción_date               datetime64[ns]
Marca_temporal_date                datetime64[ns]
Fecha_inhumación_date_diasemana          category


## 4.h Ordenar y guardar datos

Antes de guardar los datos veamos las columnas, esto nos dará una idea de que hemos hecho.

In [59]:
df.columns

Index(['Panteón_origen', 'Estatus_FC', 'Fecha_inhumación', 'Fecha_defunción',
       'Restos_tipo', 'Sexo', 'Edad', 'Conocido_Desconocido',
       'Primer apellido', 'Segundo Apellido', 'Nombre(s)', 'Nombre completo',
       'Institución_origen', 'Marca_temporal', 'Tipo_institución_origen',
       'Edad_int', 'Fecha_inhumación_date', 'Fecha_defunción_date',
       'Marca_temporal_date', 'Fecha_inhumación_date_diasemana',
       'Fecha_inhumación_date_diaaño', 'Fecha_inhumación_date_semanaaño',
       'Fecha_defunción_date_diasemana', 'Fecha_defunción_date_diaaño',
       'Fecha_defunción_date_semanaaño', 'Marca_temporal_date_diasemana',
       'Marca_temporal_date_diaaño', 'Marca_temporal_date_semanaaño'],
      dtype='object')

Al agregar columnas estas se ponen al final, por lo que sería bueno ordenarlas. Además, las columnas de fecha originales se parecen mucho a las procesadas, excepto por el cambio de formato. Entonces, podríamos quitar las originales para hacer mas pequeña la tabla. Este también es un buen momento para arrepentirse, por ejemplo quitaremos las columnas de 'diaaño'.

Dependiendo del análisis podemos quitar columnas. Por ejemplo, la columna de `Marca_temporal` y sus columnas derivadas describen cuando se hizo la captura de la información del panteón al MFC. Esto es útil si nos interesa la información administrativa del Módulo, pero no si estamos interesados en los datos demográficos de los restos en Fosa común. Por lo tanto en este caso quitaremos estas columnas.


In [60]:
df = df[['Panteón_origen', 'Estatus_FC', 
         'Fecha_inhumación_date', 'Fecha_inhumación_date_diasemana', 'Fecha_inhumación_date_semanaaño', 
         'Fecha_defunción_date', 'Fecha_defunción_date_diasemana', 'Fecha_defunción_date_semanaaño',
         'Restos_tipo', 'Sexo', 'Edad', 'Edad_int',  'Conocido_Desconocido',
         'Primer apellido', 'Segundo Apellido', 'Nombre(s)', 'Nombre completo',
         'Institución_origen', 'Tipo_institución_origen']]
df.tail()

,Panteón_origen,Estatus_FC,Fecha_inhumación_date,Fecha_inhumación_date_diasemana,Fecha_inhumación_date_semanaaño,Fecha_defunción_date,Fecha_defunción_date_diasemana,Fecha_defunción_date_semanaaño,Restos_tipo,Sexo,Edad,Edad_int,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Tipo_institución_origen
ID,,,,,,,,,,,,,,,,,,,
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,NaT,NaN,NaN,Cadáver,Masculino,42,42.0,Conocido,Flores,Rosette,Sergio,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2017-10-27,Viernes,43.0,Cadáver,Masculino,64,64.0,Conocido,Juarez,Chavez,Juan Carlos,Juan Carlos Juarez Chavez,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-08-18,Sábado,33.0,Cadáver,Masculino,NaN,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial
XX-P999,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-22,Martes,21.0,Cadáver,Femenino,1,1.0,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial
XX-P1000,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-29,Martes,22.0,Cadáver,Masculino,NaN,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial


Este es también un buen momento de cambiar los nombres de las columnas. 

**Nota** compara estos dos comandos para distinguir entre cambiar el orden de las columnas y en Nombre de estas.

In [61]:
df.columns = ['Panteón_origen', 'Estatus_FC', 
              'Fecha_inhumación', 'Fecha_inhumación_diasemana', 'Fecha_inhumación_semanaaño', 
              'Fecha_defunción', 'Fecha_defunción_diasemana', 'Fecha_defunción_semanaaño',
              'Restos_tipo', 'Sexo', 'Edad', 'Edad_años',  'Conocido_Desconocido',
              'Primer_apellido', 'Segundo_Apellido', 'Nombres', 'Nombre_completo',
              'Institución_origen', 'Tipo_institución_origen']
df.tail()

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_inhumación_diasemana,Fecha_inhumación_semanaaño,Fecha_defunción,Fecha_defunción_diasemana,Fecha_defunción_semanaaño,Restos_tipo,Sexo,Edad,Edad_años,Conocido_Desconocido,Primer_apellido,Segundo_Apellido,Nombres,Nombre_completo,Institución_origen,Tipo_institución_origen
ID,,,,,,,,,,,,,,,,,,,
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,NaT,NaN,NaN,Cadáver,Masculino,42,42.0,Conocido,Flores,Rosette,Sergio,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2017-10-27,Viernes,43.0,Cadáver,Masculino,64,64.0,Conocido,Juarez,Chavez,Juan Carlos,Juan Carlos Juarez Chavez,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-08-18,Sábado,33.0,Cadáver,Masculino,NaN,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial
XX-P999,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-22,Martes,21.0,Cadáver,Femenino,1,1.0,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial
XX-P1000,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-29,Martes,22.0,Cadáver,Masculino,NaN,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial


Hagamos un nuevo profile para ver el comportamiento de los datos limpios

In [62]:
#!pip install ydata_profiling

from ydata_profiling import ProfileReport

file_profile = "profiles/MFC_profile_clean.html"
prof = ProfileReport(df, minimal=True) 
prof.to_file(output_file=file_profile)

/home/esther/anaconda3/envs/CursoPandas2023/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Es posible guardar los datos limpios que hemos obtenido de varias formas, podemos guardarlos como csv o excel usando los comandos `.to_csv()` y `.to_excel()`. Estos formatos tienen la ventaja de que son faciles de compartir. 

Nosotros guardaremos los datos limpios en la carpeta _data_clean_ como un csv, debido a que el tamaño de la base de datos es grande, puede ser complicado para manipularla en excel. Sin embargo, es posible abrir archivos csv usando excel.

Es muy importante no rescribir los datos originales y tratar de tener una carpeta para cada parte del proceso de análisis, para evitar perder información y poder reproducir confiablemente nuestros análisis.

In [63]:
file_out = "data_clean/MFC_ActualizacionNov2022_clean.csv"
df.to_csv(file_out)

Una desventaja de guardar los archivos usando csv o excel, es que podemos perder el formato y los tipos de datos. Esto es importante sobretodo para tipos de datos como _datetime_. Una opción es guardar nuestros datos en un formato que sea facilmente interpretable para python, aunque este no se pueda trabajar con excel.

In [64]:
from joblib import dump

file_out_pickle = "data_clean/MFC_ActualizacionNov2022_clean.pkl"

with open(file_out_pickle, 'wb') as f:
    dump(df, f)

**Gracias!**